In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re, regex
from sklearn.model_selection import train_test_split

In [ ]:
df_1 = pd.read_csv('../data/raw/goemotions_1.csv')
df_2 = pd.read_csv('../data/raw/goemotions_2.csv')
df_3 = pd.read_csv('../data/raw/goemotions_3.csv')

### Clean Dataset

In [89]:
df = pd.concat([df_1, df_2, df_3], ignore_index=True)
len(df)

211225

In [90]:
df.drop_duplicates(inplace=True)
df = df[df['example_very_unclear'] == 0].drop(columns=['example_very_unclear'])
print(len(df))
df.head()

207814


,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,admiration,amusement,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Man I love reddit.,eeibobj,MrsRobertshaw,facepalm,t3_ahulml,t3_ahulml,1.547965e+09,18,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"[NAME] was nowhere near them, he was by the Fa...",eda6yn6,American_Fascist713,starwarsspeculation,t3_ackt2f,t1_eda65q2,1.546669e+09,2,0,0,...,0,0,0,0,0,0,0,0,0,1
5,Right? Considering it’s such an important docu...,eespn2i,ImperialBoss,TrueReddit,t3_aizyuz,t1_eesoak0,1.548280e+09,61,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
TARGET_EMOTIONS = ['joy', 'sadness', 'anger', 'fear', 'surprise', 'disgust', 'love', 'neutral']
keep_cols = ['text'] + TARGET_EMOTIONS
df_clear = df[keep_cols].copy()
df_clear = df_clear[df_clear[TARGET_EMOTIONS].sum(axis=1) > 0]
df_clear = df_clear.groupby('text', as_index=False)[TARGET_EMOTIONS].max()
print(df_clear.shape)
df_clear.head()

(45356, 9)


,text,joy,sadness,anger,fear,surprise,disgust,love,neutral
0,"""If you don't wear BROWN AND ORANGE...YOU DON...",0,0,1,0,0,0,0,1
1,"""What do Scottish people look like?"" How I wo...",0,0,0,0,0,0,1,0
2,"### A surprise, to be sure, but a welcome one",0,0,0,0,1,0,0,0
3,"'*Pray*, v. To ask that the laws of the unive...",0,0,0,0,0,0,0,1
4,">it'll get invaded by tankie, unfortunately. ...",0,0,0,0,0,0,0,1


In [96]:
df = df_clear.copy()

### Clean Text

In [97]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+", " ", text)           # remove URLs
    text = re.sub(r"@\w+", " ", text)                     # remove mentions
    text = regex.sub(r"[^\p{L}\p{N}\s!?.,']", " ", text)  # keep words/numbers
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
df['clean_text'] = df['text'].apply(clean_text)
df = df[df['clean_text'].str.split().str.len() >= 3]

In [104]:
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
len(train_df), len(val_df), len(test_df)

(31749, 6803, 6804)

In [106]:
train_df.to_csv("../data/train.csv", index=False)
val_df.to_csv("../data/val.csv", index=False)
test_df.to_csv("../data/test.csv", index=False)